In [34]:
import numpy as np
import pandas as pd
from torch.nn import MSELoss
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
from IPython.display import display, Math, Latex

In [35]:
a = 12
b = []
c = []
b.append(a)
c.append(a)
b[0] = 13
print(c)

[12]


In [36]:
def descriptive_length_of_fraction(numerator, denominator):
    return torch.log2((1 + torch.abs(torch.tensor(denominator))) * numerator)


def logplus(number):
    return torch.log2(1 + number**2) / 2


def descriptive_length_of_real_number(real_number, precision_floor=1e-8):
    return logplus(real_number / torch.tensor(precision_floor))

In [37]:
def FormFractionRepresentation(fraction):
    if fraction[1] != 1:
        return "\frac{" + str(fraction[0]) + "}{" + str(fraction[1]) + "}"
    return str(fraction[0])


def FormReal(number):
    return "{:.3}".format(number.item())

In [77]:
class RecursiveFormula(nn.Module):
    """
    Class used for representing formulas
    
    Attributes:
        depth
        num_variables
        powers
        lambdas - list of linear coefficients
        subformulas - list of subformulas of smaller depth, which are used for computing
        parameters - list of all learnable parameters of formula (real numbers)
        rational_values - if during simplification it turns out that some parameter is actually a rational number, 
                            its value is stored here as a tuple (numerator, denominator), otherwise there is None
        last_subformula - additional subformula, which is not multiplied by variable
    """
    
    def __init__(self, depth=0, num_variables=1):
        super(RecursiveFormula, self).__init__()
        self.depth = depth
        self.num_variables = num_variables
        self.num_parameters = 0
        self.powers = []
        self.lambdas = []
        self.subformulas = nn.ModuleList()
#         self.parameters = []
        # When depth is zero, formula is just a real number
        if depth == 0:
            new_lambda = nn.Parameter((2 * torch.randn((1, 1)))).requires_grad_(True)
            self.lambdas.append(new_lambda)
            self.register_parameter("lambda_const_{}".format(depth), new_lambda)
            self.num_parameters += 1
#             self.parameters.append(new_lambda)
        else:
            for i in range(self.num_variables):
                # When depth is 1, we do not need to create subformulas, since they would be just real numbers
                if self.depth != 1:
                    subformula = RecursiveFormula(self.depth - 1, self.num_variables)
                    self.subformulas.append(subformula)
                    self.num_parameters += subformula.num_parameters
#                     self.parameters.extend(subformula.parameters)                    
                new_lambda = nn.Parameter((2 * torch.randn((1, 1)))).requires_grad_(True)
                new_power = nn.Parameter((2 * torch.randn((1, 1)))).requires_grad_(True)
                new_rational_lambda = nn.Parameter(torch.round(new_lambda)).requires_grad_(False)
                new_rational_power = nn.Parameter(torch.round(new_power)).requires_grad_(False)
                new_rational_lambda = nn.Parameter(torch.round(new_lambda)).requires_grad_(False)
                new_rational_power = nn.Parameter(torch.round(new_power)).requires_grad_(False)
                self.register_parameter("lambda_{}".format(i), new_lambda)
                self.register_parameter("power_{}".format(i), new_power)
                self.register_parameter("rational_lambda_{}".format(i), new_rational_lambda)
                self.register_parameter("rational_power_{}".format(i), new_rational_power)
                self.lambdas.append(new_lambda)
                self.powers.append(new_power)
                self.num_parameters += 2
#                 self.parameters.extend([new_lambda, new_power])
            self.last_subformula = RecursiveFormula(self.depth - 1, self.num_variables)
            self.num_parameters += self.last_subformula.num_parameters
#             self.parameters.extend(self.last_subformula.parameters)
        # Before simplification we assume that all parameters are real
        self.rational_lambdas = [None for lambd in self.lambdas]
        self.rational_powers = [None for power in self.powers]
        
                                    
    def forward(self, x):
        """
        Iterate over subformulas, recursively computing result using results of subformulas
        """
        # When depth is 0, we just return the corresponding number
        if self.depth == 0:
            return self.lambdas[0].repeat(x.shape[0], 1)
        
        ans = torch.zeros(x.shape[0], 1)
        for i in range(self.num_variables):
            x_powered = torch.t(x[:, i]**self.powers[i])
            subformula_result = torch.ones((x.shape[0], 1))
            # When depth is 1, we do not need to compute subformulas
            if self.depth != 1:
                subformula_result = self.subformulas[i](x)
            assert subformula_result.shape == (x.shape[0], 1)
            assert x_powered.shape == (x.shape[0], 1)
            ans += self.lambdas[i] * x_powered * subformula_result           
        ans += self.last_subformula(x)
        return ans
    
    def simplify(self, X_val, y_val, max_denominator=1, inplace=False):
        simplified_version = copy.deepcopy(self)  
        simplified_state_dict = simplified_version.state_dict()
        # Iterate over all parameters, try to substitute them with some rational number.
        for key, value in self.state_dict():
            simplified_version_for_iteration = copy.deepcopy(simplified_version)
            simplified_state_dict_for_iteration = simplified_version_for_iteration.state_dict()
            y_predict = simplified_version(X_val)
            loss = MSELoss()(y_val, y_predict)
            descriptive_length_of_loss = descriptive_length_of_real_number(loss)
            descriptive_length_of_existing_parameter = descriptive_length_of_real_number(parameter)
            # Iterate over all possible denominators
            for possible_denominator in range(1, max_denominator + 1):
                print("trying denominator", possible_denominator)
                simplified_parameter_numerator = torch.round(parameter * possible_denominator).item()
                simplified_state_dict_for_iteration[key] = simplified_parameter_numerator / possible_denominator
                simplified_version_for_iteration.load_state_dict(simplified_state_dict_for_iteration)
                descriptive_length_of_simplified_parameter = descriptive_length_of_fraction(simplified_parameter_numerator, possible_denominator)
                y_predict_simplified = simplified_version_for_iteration(X_val)
                loss_of_simplified_model = MSELoss()(y_val, y_predict_simplified)
                descriptive_length_of_loss_of_simplified_model = descriptive_length_of_real_number(loss_of_simplified_model)                
                # If the descriptive length did not improve, revert the change.
                if descriptive_length_of_loss_of_simplified_model + descriptive_length_of_simplified_parameter > descriptive_length_of_loss + descriptive_length_of_existing_parameter:
                    simplified_version_for_iteration.load_state_dict(simplified_state_dict)
                
            simplified_state_dict = simplified_state_dict_for_iteration
            simplified_version.load_state_dict(simplified_state_dict)
        
        if inplace:
            self = copy.deepcopy(simplified_version)
        else:
            return simplified_version      
    
    def get_lambda(self, i):
        return self.state_dict['lambda_'.format(i)]
    
    def get_rational_lambda(self, i):
        return self.state_dict['rational_lambda_'.format(i)]
    
    def get_power(self, i):
        return self.state_dict['power_'.format(i)]
    
    def get_rational_power(self, i):
        return self.state_dict['rational_power_'.format(i)]
    
    def __repr__(self):
        """
        Return tex-style string, recursively combining result from representation of subformulas
        """
        if self.depth == 0:
            if self.self.get_rational_lambda(0) is not None:
                return FormFractionRepresentation(self.get_rational_lambda(0))            
            return FormReal(self.get_lambda(0))
        
        ans = ["("]
        for i in range(self.num_variables):
            if i != 0 and self.get_lambda(i) > 0:
                ans.append(" + ")
            if self.get_rational_lambda(i) is not None:
                ans.append(FormFractionRepresentation(get_rational_lambda(i))
            else:
                ans.append(FormReal(self.get_lambda(i))   
            
            ans.append("x_{}^".format(i + 1) + "{")
            if self.get_rational_power(i) is not None:
                ans.append(FormFractionRepresentation(get_rational_power(i))
            else:
                ans.append(FormReal(self.get_power(i))  
            ans += "}"    
            if self.depth != 1:
                ans.append(str(self.subformulas[i]))
        if self.last_subformula.lambdas[0] > 0:        
            ans.append(" + ")
        ans.append(str(self.last_subformula))
        ans.append(")")
        ans = ''.join(ans)
        return ans

In [78]:
def info(formula):
    print("depth: {}, number of variables: {}, total parameters: {}".format(
        formula.depth, formula.num_variables, len(formula.parameters)))
    
def PrintFormula(formula, mode="slow"):
#     info(network)
    if mode == "slow":
        display(Math(str(formula)))   
    else:
        print(formula)

In [79]:
def LearnFormula(X, y, n_init=2, max_iter=5000, depth=2, verbose=2, max_epochs_without_improvement=500,
                minimal_acceptable_improvement=3e-6):
    best_model = RecursiveFormula(depth, X.shape[1])
    best_loss = 1e10
    for initiation in range(n_init):
    #     torch.random.manual_seed(seed)
        formula = RecursiveFormula(depth, X.shape[1])
        # create your optimizer
        optimizer = optim.Rprop(formula.parameters(), lr=1e-3)
        criterion = nn.MSELoss()
        epochs_without_improvement = 0
        epoch = 0
        optimizer.zero_grad()
        output = formula(X)
        previous_loss = criterion(output, y).item() 
        while epoch < max_iter and epochs_without_improvement < max_epochs_without_improvement:
            optimizer.zero_grad()
            output = formula(X)
            loss = criterion(output, y) 
            loss.backward()
            if verbose and (epoch + 1) % 500 == 0:
                print("Epoch {}, current loss {:.3}, current formula ".format(epoch + 1, loss.item()), end='')
                PrintFormula(formula, "fast")       
            optimizer.step()  
            epoch += 1
            if torch.abs(previous_loss - loss) < minimal_acceptable_improvement:
                epochs_without_improvement += 1
#                 print(torch.abs(previous_loss - loss))
            else:
                epochs_without_improvement = 0
            previous_loss = loss.item()
        if loss < best_loss:
            best_loss = loss
            best_formula = formula
    return best_formula

In [74]:
X1 = torch.rand(100, 1) * 10
y1 = 2.5 * X1**2 + 3

X2 = torch.rand(100, 2) * 10
y2 = 2.5 * X2[:, 0]**2 + 0.333 * X2[:, 1]**0.5 + 3    
# y = 1.2 * X[:, 0]**2.1 * X[:, 2] + 2.5 * X[:, 1]**(-3) + 1/2 * X[:, 2]**0.3333 * X[:, 1]**(-4)

In [80]:
formula = RecursiveFormula(3, 3)

In [81]:
formula.state_dict()

OrderedDict([('lambda_0', tensor([[-0.0309]])),
             ('power_0', tensor([[-2.2975]])),
             ('rational_lambda_0', tensor([[-0.]])),
             ('rational_power_0', tensor([[-2.]])),
             ('lambda_1', tensor([[3.3882]])),
             ('power_1', tensor([[-2.1302]])),
             ('rational_lambda_1', tensor([[3.]])),
             ('rational_power_1', tensor([[-2.]])),
             ('lambda_2', tensor([[-1.9455]])),
             ('power_2', tensor([[-0.3782]])),
             ('rational_lambda_2', tensor([[-2.]])),
             ('rational_power_2', tensor([[-0.]])),
             ('subformulas.0.lambda_0', tensor([[-1.6850]])),
             ('subformulas.0.power_0', tensor([[-1.7232]])),
             ('subformulas.0.rational_lambda_0', tensor([[-2.]])),
             ('subformulas.0.rational_power_0', tensor([[-2.]])),
             ('subformulas.0.lambda_1', tensor([[0.7629]])),
             ('subformulas.0.power_1', tensor([[-1.4637]])),
             ('subformul

In [65]:
formula.subformulas[0].state_dict()

OrderedDict([('lambda_0', tensor([[0.0831]])),
             ('power_0', tensor([[-0.3652]])),
             ('lambda_1', tensor([[-0.3794]])),
             ('power_1', tensor([[0.0476]])),
             ('last_subformula.lambda_const_0', tensor([[-2.4612]]))])

In [24]:
formula.parameters

[Parameter containing:
 tensor([[2.4957]], requires_grad=True), Parameter containing:
 tensor([[2.0008]], requires_grad=True), Parameter containing:
 tensor([[3.0192]], requires_grad=True)]

In [25]:
simplified_formula = formula.simplify(X1, y1, inplace=False)

trying denominator 1
trying denominator 1
trying denominator 1


In [26]:
simplified_formula.rational_values

[(2, 1), (2, 1), (3, 1)]

In [16]:
a = RecursiveFormula(3, 3)

In [30]:
old_dict = a.state_dict()
old_dict['lambda_0'] = torch.tensor([[179]])
a.load_state_dict(old_dict)

<All keys matched successfully>

In [31]:
a.state_dict()

OrderedDict([('lambda_0', tensor([[179.]])),
             ('power_0', tensor([[0.3915]])),
             ('lambda_1', tensor([[0.2409]])),
             ('power_1', tensor([[0.3700]])),
             ('lambda_2', tensor([[-0.0161]])),
             ('power_2', tensor([[-0.7756]])),
             ('subformulas.0.lambda_0', tensor([[2.2854]])),
             ('subformulas.0.power_0', tensor([[-2.8714]])),
             ('subformulas.0.lambda_1', tensor([[1.1982]])),
             ('subformulas.0.power_1', tensor([[-0.9329]])),
             ('subformulas.0.lambda_2', tensor([[-2.0648]])),
             ('subformulas.0.power_2', tensor([[-2.5893]])),
             ('subformulas.0.subformulas.0.lambda_0', tensor([[-0.2927]])),
             ('subformulas.0.subformulas.0.power_0', tensor([[-1.1151]])),
             ('subformulas.0.subformulas.0.lambda_1', tensor([[0.2732]])),
             ('subformulas.0.subformulas.0.power_1', tensor([[-3.8089]])),
             ('subformulas.0.subformulas.0.lambda_2',

In [14]:
PrintFormula(LearnFormula(X2, y2))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:431: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 500, current loss 5.07e+03, current formula (2.62x_1^{2.58e-06} + 1.89x_2^{1.88e-06} + 86.9)
Epoch 500, current loss 5.07e+03, current formula (1.84x_1^{-5.03e-06} + 5.41x_2^{1.9e-06} + 84.2)


<IPython.core.display.Math object>

In [15]:
MSELoss()(y1, y1)

tensor(0.)

In [16]:
for i in formula.parameters():
    print(i)

TypeError: 'list' object is not callable